# The Basics

In [ ]:
# Import Packages
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns
import networkx as nx

import geopandas

%matplotlib inline
pd.set_option('display.max_rows', 999)

# **Exploratory Data Analysis**

In [ ]:
# read in data
covid_f_raw = pd.read_csv('data/covid_forecast_2020-12-03.csv')
covid_h_raw = pd.read_csv('data/COVID_historical_us-counties.csv',
                         converters={'fips': lambda x: str(x)})

census_raw = pd.read_csv('data/2010_census_pop.csv',
                    converters={'Zip Code ZCTA': lambda x: str(x)})
pop_den_raw = pd.read_csv('data/uszips_pop_density.csv',
                     converters={'zip': lambda x: str(x)})
fips_pop = pd.read_csv('data/fips_population.csv',
                      converters={'FIPS': lambda x: str(x)})

fb = pd.read_table('data/county_county_aug2020.tsv', 
                   converters={'user_loc': lambda x: str(x), 
                               'fr_loc': lambda x: str(x)})

# Generate Map File from Shape File
us_map = geopandas.read_file('data/cb_2018_us_county_20m 2/cb_2018_us_county_20m.shp')

# Log Transform 'Scaled SCI' 
fb['log_sci'] = np.log10(fb['scaled_sci'])

In [ ]:
sf_county = fb[fb['user_loc'] == '06075']
sf_county['GEOID'] = sf_county['fr_loc']

kern_county = fb[fb['user_loc'] == '06029']
kern_county['GEOID'] = kern_county['fr_loc']

In [ ]:
sf_map = us_map.merge(sf_county, on='GEOID')
kern_map = us_map.merge(kern_county, on='GEOID')



In [ ]:
sf_map

## GIS Exploration of County Social Connections

In [ ]:
# plot Social Connections of San Francisco County residents to other counties

fig, ax = plt.subplots(figsize=(20, 15))
sf_map.plot(ax=ax, column='log_sci', 
            scheme='natural_breaks', 
            cmap='Blues', 
            legend=True)

ax.set_xlim(-125, -65)
ax.set_ylim(23, 50)
ax.set(title="San Francisco County")
ax.axis("off")
# plt.savefig('SF_county_SCI_map.png')
;

In [ ]:
# plot Social Connections of Kern County residents to other counties

fig, ax = plt.subplots(figsize=(20, 15))
kern_map.plot(ax=ax, column='log_sci', 
            scheme='natural_breaks', 
            cmap='Blues', 
            legend=True)
# leg = ax.get_legend()
ax.set_xlim(-125, -65)
ax.set_ylim(23, 50)
ax.set(title="Kern County")
ax.axis("off")
plt.savefig('Kern_county_SCI_map.png');

## Other Data Visualizations

In [ ]:
# Histogram of Log SCI county pairs

log_sci_hist = fb['log_sci']
log_sci_hist = log_sci_hist.where(log_sci_hist > 0)

f, ax = plt.subplots(figsize=(12, 5))
sns.despine(f)

sns.histplot(log_sci_hist, 
             color='#CB2A52'
            )

sns.set_style("white")

ax.set_ylim(0, 60000)
ax.set_xlim(0, 7)
ax.set_xlabel('log, SCI')
ax.set_title('Distribution of County-County Pairs by log SCI',
            size=18)
plt.savefig('log_SCI_distribution.png', dpi=1200)
;

In [ ]:
# Box Plot of Log SCI county pairs

plt.boxplot(fb['log_sci'])
ax.ticklabel_format(useOffset=False, style='plain');

# Graph Analysis

## Data Cleaning

In [ ]:
fb_ga = fb.copy()
fb_ga

In [ ]:
# create fb_ga quartiles for lower middle, and upper quartiles
fb_ga.loc[fb_ga['log_sci'] <= fb_ga['log_sci'].quantile(.25), 'SCI_lower_25'] = '1'
fb_ga.loc[fb_ga['log_sci'] >= fb_ga['log_sci'].quantile(.75), 'SCI_upper_25'] = '1'
fb_ga.loc[(fb_ga['log_sci'] > fb_ga['log_sci'].quantile(.25)) & \
           (fb_ga['log_sci'] < fb_ga['log_sci'].quantile(.75)) , \
           'SCI_middle_50'] = '1'

fb_ga_lower_25 = fb_ga[fb_ga.SCI_lower_25 == '1']
fb_ga_middle_50 = fb_ga[fb_ga.SCI_middle_50 == '1']
fb_ga_upper_25 = fb_ga[fb_ga.SCI_upper_25 == '1']

In [ ]:
# Cleaning COVID datasets
na_covid_cases = covid_h_raw.groupby('fips').max()
na_covid_cases.reset_index(inplace=True)
na_covid_cases.replace("", np.nan, inplace=True)
na_covid_cases.dropna(inplace=True)

na_fips_pop_covid = fips_pop.set_index('FIPS').join(na_covid_cases.set_index('fips')).dropna()
na_fips_pop_covid['cases_per_capita'] = na_fips_pop_covid['cases']/1000
na_fips_pop_covid.sort_values('cases_per_capita').reset_index()

## San Francisco, network

In [ ]:
sf_county = fb[fb['user_loc'] == '06075']

In [ ]:
g_sf = nx.from_pandas_edgelist(sf_county, "user_loc", "fr_loc", ["scaled_sci", "log_sci"])
print('# of edges: {}'.format(g_sf.number_of_edges()))
print('# of nodes: {}'.format(g_sf.number_of_nodes()))

In [ ]:
lower_25_edges_sample = fb_ga_lower_25.sample(n=2000)
lower_25_edges_sample.drop(labels=['SCI_lower_25', 'SCI_upper_25', 'SCI_middle_50'], axis=1, inplace=True)

g_lower_25_sample = nx.from_pandas_edgelist(lower_25_edges_sample, "user_loc", "fr_loc", ["scaled_sci", "log_sci"])
print('# of edges: {}'.format(g_lower_25_sample.number_of_edges()))
print('# of nodes: {}'.format(g_lower_25_sample.number_of_nodes()))

In [ ]:
# reset index of na_fips_pop_covid
na_fips_pop_covid = na_fips_pop_covid.reset_index()

# create nodelist from popdensci_join
nodelist = na_fips_pop_covid.copy()
nodelist = nodelist[['FIPS', 'cases_per_capita']]

# convert node attributes in nodelist to dict
node_attributes_sf = dict(zip(nodelist.FIPS, nodelist.cases_per_capita))

## Kern County, Network

In [ ]:
kern_county = fb[fb['user_loc'] == '06029']

g_kern = nx.from_pandas_edgelist(kern_county, "user_loc", "fr_loc", ["scaled_sci", "log_sci"])
print('# of edges: {}'.format(g_kern.number_of_edges()))
print('# of nodes: {}'.format(g_kern.number_of_nodes()))

# reset index of na_fips_pop_covid
# na_fips_pop_covid = na_fips_pop_covid.reset_index()

# create nodelist from popdensci_join
nodelist = na_fips_pop_covid.copy()
nodelist = nodelist[['FIPS', 'cases_per_capita']]

# convert node attributes in nodelist to dict
node_attributes_kern = dict(zip(nodelist.FIPS, nodelist.cases_per_capita))

## SCI Quartile Exploration

### Lower 25%

In [ ]:
lower_25_edges_sample = fb_ga_lower_25.sample(n=2000)
lower_25_edges_sample.drop(labels=['SCI_lower_25', 'SCI_upper_25', 'SCI_middle_50'], axis=1, inplace=True)

g_lower_25_sample = nx.from_pandas_edgelist(lower_25_edges_sample, "user_loc", "fr_loc", ["scaled_sci", "log_sci"])
print('# of edges: {}'.format(g_lower_25_sample.number_of_edges()))
print('# of nodes: {}'.format(g_lower_25_sample.number_of_nodes()))

In [ ]:
# reset index of na_fips_pop_covid
# na_fips_pop_covid = na_fips_pop_covid.reset_index()

# create nodelist from popdensci_join
node_attributes_lower_25_sample = na_fips_pop_covid.copy()
node_attributes_lower_25_sample = node_attributes_lower_25_sample[['FIPS', 'cases_per_capita']]

# deduplicate nodelists by joining edgelist to it
node_attributes_lower_25_sample = lower_25_edges_sample.set_index('user_loc').join(node_attributes_lower_25_sample.set_index('FIPS'))

# convert node attributes in nodelist to dict
node_attributes_lower_25_sample = dict(zip(node_attributes_lower_25_sample.fr_loc, node_attributes_lower_25_sample.cases_per_capita))

In [ ]:
# Lower 25 Percentile

plt.figure(figsize=(15, 12))
# pos = nx.kamada_kawai_layout(g)
nx.draw(g_lower_25_sample,
        nodelist=node_attributes_lower_25_sample.keys(), 
        node_size=[v*5 for v in node_attributes_lower_25_sample.values()], 
        with_labels=False, 
        font_size=10, 
        width= [data['log_sci']/10 for node1, node2, data in g_lower_25_sample.edges(data=True)], 
        linewidths=0,
        node_color = '#DA4167',
        alpha=0.65)

# plt.savefig('network_map_lower_50.png', dpi=1200)
# plt.show()

### Middle 50%

In [ ]:
middle_50_edges_sample = fb_ga_middle_50.sample(n=2000)
middle_50_edges_sample.drop(labels=['SCI_lower_25', 'SCI_upper_25', 'SCI_middle_50'], axis=1, inplace=True)

g_middle_50_sample = nx.from_pandas_edgelist(middle_50_edges_sample, "user_loc", "fr_loc", ["scaled_sci", "log_sci"])
print('# of edges: {}'.format(g_middle_50_sample.number_of_edges()))
print('# of nodes: {}'.format(g_middle_50_sample.number_of_nodes()))

In [ ]:
# reset index of na_fips_pop_covid
# na_fips_pop_covid = na_fips_pop_covid.reset_index()

# create nodelist from popdensci_join
node_attributes_middle_50_sample = na_fips_pop_covid.copy()
node_attributes_middle_50_sample = node_attributes_middle_50_sample[['FIPS', 'cases_per_capita']]

# deduplicate nodelists by joining edgelist to it
node_attributes_middle_50_sample = middle_50_edges_sample.set_index('user_loc').join(node_attributes_middle_50_sample.set_index('FIPS'))

# convert node attributes in nodelist to dict
node_attributes_middle_50_sample = dict(zip(node_attributes_middle_50_sample.fr_loc, node_attributes_middle_50_sample.cases_per_capita))

In [ ]:
plt.figure(figsize=(15, 12))
# pos = nx.kamada_kawai_layout(g)
nx.draw(g_middle_50_sample,
        nodelist=node_attributes_middle_50_sample.keys(), 
        node_size=[v*5 for v in node_attributes_middle_50_sample.values()], 
        with_labels=False, 
        font_size=10, 
        width= [data['log_sci']/10 for node1, node2, data in g_middle_50_sample.edges(data=True)], 
        linewidths=0,
        node_color='#DA4167',
        alpha=0.65)
# plt.savefig('network_map_middle_50.png', dpi=1200)
# plt.show();

### Upper 25%

In [ ]:
upper_25_edges_sample = fb_ga_upper_25.sample(n=2000)
upper_25_edges_sample.drop(labels=['SCI_lower_25', 'SCI_upper_25', 'SCI_middle_50'], axis=1, inplace=True)

g_upper_25_sample = nx.from_pandas_edgelist(upper_25_edges_sample, "user_loc", "fr_loc", ["scaled_sci", "log_sci"])
print('# of edges: {}'.format(g_upper_25_sample.number_of_edges()))
print('# of nodes: {}'.format(g_upper_25_sample.number_of_nodes()))

# reset index of na_fips_pop_covid
# na_fips_pop_covid = na_fips_pop_covid.reset_index()

# create nodelist from popdensci_join
node_attributes_upper_25_sample = na_fips_pop_covid.copy()
node_attributes_upper_25_sample = node_attributes_upper_25_sample[['FIPS', 'cases_per_capita']]

# deduplicate nodelists by joining edgelist to it
node_attributes_upper_25_sample = upper_25_edges_sample.set_index('user_loc').join(node_attributes_upper_25_sample.set_index('FIPS'))

# convert node attributes in nodelist to dict
node_attributes_upper_25_sample = dict(zip(node_attributes_upper_25_sample.fr_loc, node_attributes_upper_25_sample.cases_per_capita))

In [ ]:
plt.figure(figsize=(15, 12))
# pos = nx.kamada_kawai_layout(g)
nx.draw(g_upper_25_sample,
        nodelist=node_attributes_upper_25_sample.keys(), 
        node_size=[v*5 for v in node_attributes_upper_25_sample.values()], 
        with_labels=False, 
        font_size=10, 
        width= [data['log_sci']/10 for node1, node2, data in g_upper_25_sample.edges(data=True)], 
        linewidths=0,
        node_color='#DA4167',
        alpha=0.75)

# plt.show()
# plt.savefig('network_map_upper_25.png', dpi=1200)

# Hypothesis Testing

After taking a sample of upper SCI quartile counties, is the average number of COVID cases per capita significantly greater than the population mean?

1. Formulate hypotheses

$H_0$: High SCI counties have no difference in number of covid cases/capita than lower SCI counties.

$H_a$: High SCI counties have a significantly different number of covid cases/capita than lower SCI counties.

2. Choose a significance level

$\alpha = 0.05$ 

3. Choose a statistical test, find the test statistic

- two tailed t test

- $\bar{X}$ = 

4. Compute probability of results assuming the null hypothesis is true

- outputs the p value

5. Compare p-value to alpha to draw conclusion

- p <= alpha, reject null in favor of the alternative
- p > alpha, fail to reject the null


3a. calculate sample mean from size n, calculate sample standard deviation

3b. calculate z statistic - 
sample mean - population mean / (pop standard deviation / sqroot of n)



In [ ]:
# upper

# sample from upper SCI counties
ht_ga_upper_25_sample = fb_ga_upper_25.sample(n=300)

# drop duplicates
ht_ga_upper_25_sample = ht_ga_upper_25_sample.drop_duplicates('user_loc')

# container of sample upper quartile SCI counties + COVID/capita
ht_ga_upper_25_sample = ht_ga_upper_25_sample.set_index('user_loc').join(na_fips_pop_covid.set_index('FIPS'))

# drop nas
ht_ga_upper_25_sample_ttest = ht_ga_upper_25_sample['cases_per_capita'].dropna()

# container of average cases/capita
ht_ga_upper_25_sample_ttest.count()

In [ ]:
# lower

# sample from lower SCI counties
ht_ga_lower_25_sample = fb_ga_lower_25.sample(n=300)

# drop duplicates
ht_ga_lower_25_sample = ht_ga_lower_25_sample.drop_duplicates('user_loc')

# container of sample lower quartile SCI counties + COVID/capita
ht_ga_lower_25_sample = ht_ga_lower_25_sample.set_index('user_loc').join(na_fips_pop_covid.set_index('FIPS'))

# drop nas
ht_ga_lower_25_sample_ttest = ht_ga_lower_25_sample['cases_per_capita'].dropna()

# container of average cases/capita
ht_ga_lower_25_sample_ttest.count()

In [ ]:
# two-tailed t test

from scipy import stats
stats.ttest_ind(ht_ga_lower_25_sample_ttest, ht_ga_upper_25_sample_ttest, axis=0, nan_policy='propagate')

In [ ]:
# population mean COVID cases/capita
pop_mean = na_fips_pop_covid['cases_per_capita'].mean()
print(f"Population mean: {na_fips_pop_covid['cases_per_capita'].mean():.2f}")

# population standard deviation COVID cases/capita
pop_std = na_fips_pop_covid['cases_per_capita'].std()
print(f"Population standard deviation: {na_fips_pop_covid['cases_per_capita'].std():.2f}")

# sample mean of COVID cases/capita
sample_mean = ht_ga_upper_25_sample['cases_per_capita'].mean()
print(f"Sample mean: {ht_ga_upper_25_sample['cases_per_capita'].mean():.2f}")

# sample standard deviation
sample_std = ht_ga_upper_25_sample['cases_per_capita'].std()
print(f"Sample standard deviation: {ht_ga_upper_25_sample['cases_per_capita'].std():.2f}")

# z score
z_score = sample_mean-pop_mean/(pop_std/np.sqrt(300))
print(f"z score: {sample_mean-pop_mean/(pop_std/np.sqrt(300)):.2f}")

# p value
import scipy.stats as st
print(f"p-value: {st.norm.pdf(z_score):.2f}")